## Ablation Analysis

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
import tqdm

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
from tardis._utils.functions import create_random_mask
import sklearn
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
tardis.config = tardis.config_server

In [3]:
torch.cuda.is_available()

True

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [5]:
adata_file_path = os.path.join(tardis.config.io_directories["processed"], "biolord_afriat.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)
adata

AnnData object with n_obs × n_vars = 19053 × 5000
    obs: 'mouse', 'experiment', 'time_int', 'time_cat', 'zone', 'status_control'

In [6]:
latent_dict = dict()
reconst_r2_all = dict()
reconst_r2_valid = dict()
mi_normalized = dict()
max_mig_all = dict()
max_mig_valid = dict()

In [7]:
bs = 16
calcs = sorted([
    0.0, 
    0.2, 0.6, 
    0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 
    0.991, 0.992, 0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999
])

In [8]:
adata = ad.read_h5ad(adata_file_path)
ratio_true=0.0
mask = create_random_mask(adata.shape[1], ratio_true=ratio_true, seed=2024)
adata.X[:, mask] = 0.0

dir_path = os.path.join(
    tardis.config.io_directories["models"],
    "afriat_scvi"
)
vae = tardis.MyModel.load(dir_path=dir_path, adata=adata)
latent_dict["scvi"] = vae.get_latent_representation()

for tensors in vae._make_data_loader(adata=adata, batch_size=adata.n_obs):
    pred = (
        vae.module.my_sample(tensors=tensors, latent_space=False, return_dist=False, n_samples=1)
        .cpu()
        .numpy()
        .mean(axis=0)  # aggregate 'n_sample's
    )

ll = []
for i in range(int(adata.shape[0]/bs)-1):
    indices = list(range(bs * i, bs * (i + 1)))
    true = adata.X[indices, :][:, ~mask].mean(axis=0)
    _pred = pred[indices, :][:, ~mask].mean(axis=0)
    ll.append(sklearn.metrics.r2_score(true, _pred))
reconst_r2_all["scvi"] = np.mean(ll)

ll = []
vi = vae.validation_indices
for i in range(int(len(vi)/bs)-1):
    indices_ = list(range(bs * i, bs * (i + 1)))
    indices = vi[indices_]
    true = adata.X[indices, :][:, ~mask].mean(axis=0)
    _pred = pred[indices, :][:, ~mask].mean(axis=0)
    ll.append(sklearn.metrics.r2_score(true, _pred))
reconst_r2_valid["scvi"] = np.mean(ll)

INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/afriat_scvi/model.pt already downloaded


CUDA backend failed to initialize: Found cuDNN version 8700, but JAX was built against version 8907, which is newer. The copy of cuDNN that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/icb/kemal.inecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [9]:
for ratio_true in tqdm.tqdm(calcs):
    adata = ad.read_h5ad(adata_file_path)
    mask = create_random_mask(adata.shape[1], ratio_true=ratio_true, seed=2024)
    adata.X[:, mask] = 0.0
    
    dir_path = os.path.join(
        tardis.config.io_directories["models"],
        ("afriat_v2_4" if ratio_true == 0.0 else f"afriat_v2_4_ablation_{ratio_true}")
    )
    vae = tardis.MyModel.load(dir_path=dir_path, adata=adata)
    latent_dict[ratio_true] = vae.get_latent_representation()
    
    for tensors in vae._make_data_loader(adata=adata, batch_size=adata.n_obs):
        pred = (
            vae.module.my_sample(tensors=tensors, latent_space=False, return_dist=False, n_samples=1)
            .cpu()
            .numpy()
            .mean(axis=0)  # aggregate 'n_sample's
        )
    
    ll = []
    for i in range(int(adata.shape[0]/bs)-1):
        indices = list(range(bs * i, bs * (i + 1)))
        true = adata.X[indices, :][:, ~mask].mean(axis=0)
        _pred = pred[indices, :][:, ~mask].mean(axis=0)
        ll.append(sklearn.metrics.r2_score(true, _pred))
    reconst_r2_all[ratio_true] = np.mean(ll)
    
    ll = []
    vi = vae.validation_indices
    for i in range(int(len(vi)/bs)-1):
        indices_ = list(range(bs * i, bs * (i + 1)))
        indices = vi[indices_]
        true = adata.X[indices, :][:, ~mask].mean(axis=0)
        _pred = pred[indices, :][:, ~mask].mean(axis=0)
        ll.append(sklearn.metrics.r2_score(true, _pred))
    reconst_r2_valid[ratio_true] = np.mean(ll)
    
    mi_normalized[ratio_true] = {z: vae.get_MI_normalized_training(z) for z in ["zone", "status_control", "time_cat"]}

    from tardis._disentanglementmanager import DisentanglementManager as DM
    factors_indices = sorted([[indice, config.obs_key] for config in DM.configurations.items for indice in config.reserved_latent_indices])
    factors = np.vstack([sklearn.preprocessing.LabelEncoder().fit_transform(
        adata.obs[j].copy().astype(str).values.flatten()
    ) for i, j in factors_indices]).T
    data = latent_dict[ratio_true].copy()[:, [i for i, j in factors_indices]]
    max_mig_all[ratio_true] = np.mean(vae.get_MI_precalculated(data=data, factors=factors, variation="maxMIG", discretization_bins=256))
    max_mig_valid[ratio_true] = np.mean(vae.get_MI_precalculated(data=data[vae.validation_indices, :], factors=factors[vae.validation_indices, :], variation="maxMIG", discretization_bins=256))

  0%|                                                                                                                                                                                                                                                                     | 0/22 [00:00<?, ?it/s]

INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/afriat_v2_4/model.pt already downloaded


/home/icb/kemal.inecik/tools/apps/mamba/envs/tardis_env/lib/python3.10/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
  5%|███████████▌                                                                                                                                                                                                                                                 | 1/22 [00:07<02:42,  7.73s/it]

INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/afriat_v2_4_ablation_0.2/model.pt      
         already downloaded                                                                                        


  9%|███████████████████████                                                                                                                                                                                                                                      | 2/22 [00:15<02:33,  7.69s/it]

...

/home/icb/kemal.inecik/work/codes/tardis/tardis/_mymodel.py:81: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(self.adata_manager, n_batch)
 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                       | 20/22 [02:26<00:14,  7.26s/it]

INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/afriat_v2_4_ablation_0.998/model.pt    
         already downloaded                                                                                        


/home/icb/kemal.inecik/work/codes/tardis/tardis/_mymodel.py:81: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(self.adata_manager, n_batch)
 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌           | 21/22 [02:33<00:07,  7.19s/it]

INFO     File /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/models/afriat_v2_4_ablation_0.999/model.pt    
         already downloaded                                                                                        


/home/icb/kemal.inecik/work/codes/tardis/tardis/_mymodel.py:81: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(self.adata_manager, n_batch)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [02:40<00:00,  7.28s/it]


In [10]:
max_mig_valid

{0.0: 0.8583506330202505,
 0.2: 0.8498139935860906,
 0.6: 0.8390536713388382,
 0.9: 0.8536516019679349,
 0.91: 0.8005643443602674,
 0.92: 0.7998798752617361,
 0.93: 0.7278730959170586,
 0.94: 0.796495055437099,
 0.95: 0.7677566834682108,
 0.96: 0.7288645187768626,
 0.97: 0.6958833527351809,
 0.98: 0.6888875243125874,
 0.99: 0.622978087924154,
 0.991: 0.6165318418068416,
 0.992: 0.6309514400606709,
 0.993: 0.5539609599907348,
 0.994: 0.5662787223068028,
 0.995: 0.5611022679779271,
 0.996: 0.5088957628474783,
 0.997: 0.5177442520083118,
 0.998: 0.5236418560889778,
 0.999: 0.3937502151886305}

In [11]:
{i: v["zone"][('validation', 'unreserved')] for i, v in mi_normalized.items()}

{0.0: 0.09694134593558497,
 0.2: 0.09715021880345759,
 0.6: 0.10019364663962538,
 0.9: 0.13463923252920243,
 0.91: 0.16289691655695382,
 0.92: 0.1434848684972842,
 0.93: 0.14028619442259935,
 0.94: 0.14761419764318343,
 0.95: 0.1645764317632637,
 0.96: 0.20394805364821064,
 0.97: 0.17967801813066595,
 0.98: 0.21844107384079142,
 0.99: 0.3416248360435271,
 0.991: 0.25878338496930187,
 0.992: 0.296693641101551,
 0.993: 0.40985474953823164,
 0.994: 0.36004423123247,
 0.995: 0.4365020151301172,
 0.996: 0.5000249220542329,
 0.997: 0.5860658303887643,
 0.998: 0.5105868744009107,
 0.999: 0.7971474412140888}

In [12]:
reconst_r2_all, reconst_r2_valid

({'scvi': 0.9794915922725316,
  0.0: 0.9776903543918521,
  0.2: 0.9781409913967698,
  0.6: 0.9764483871375248,
  0.9: 0.954094042430282,
  0.91: 0.9493476993149418,
  0.92: 0.9575664963591353,
  0.93: 0.9323883094050397,
  0.94: 0.9476973628643781,
  0.95: 0.9118750722140906,
  0.96: 0.919130124857641,
  0.97: 0.9397611331392058,
  0.98: 0.8293069416335367,
  0.99: 0.8599047332993995,
  0.991: 0.8741459984794296,
  0.992: 0.8486046412693782,
  0.993: 0.892134242693435,
  0.994: 0.8845538985344187,
  0.995: 0.8933276341262962,
  0.996: 0.9139460951743921,
  0.997: 0.9488919048544922,
  0.998: 0.9424651596012371,
  0.999: 0.5651373344126887},
 {'scvi': 0.9760289868776779,
  0.0: 0.9748477564208388,
  0.2: 0.9755500219128085,
  0.6: 0.9753313509864432,
  0.9: 0.971821723123256,
  0.91: 0.9703304177570348,
  0.92: 0.9756216857337611,
  0.93: 0.9611084955535352,
  0.94: 0.9731458529292131,
  0.95: 0.9667928509020741,
  0.96: 0.9683855734805639,
  0.97: 0.9793984037118519,
  0.98: 0.91674127

# Cluster distribution

In [13]:
adata = ad.read_h5ad(adata_file_path)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
sc.pp.pca(adata)

for k, v in latent_dict.items():
    adata.obsm[str(k)] = v

In [15]:
ll = dict()
obs_key = "time_cat"
for obsm in ['0.0', '0.2', '0.6', '0.9', '0.91', '0.92', '0.93', '0.94', '0.95', '0.96', '0.97', '0.98', '0.99', '0.991', '0.992', '0.993', '0.994', '0.995', '0.996', '0.997', '0.998', '0.999']:
    ind = DM.configurations.get_ordered_obs_key().index(obs_key)
    data = adata.obsm[str(obsm)][:, list(range(8 * ind, 8 * (1 + ind)))]
    l = []
    for c in adata.obs["time_cat"].unique():
        cat_data = data[(adata.obs["time_cat"] == c), :]
        l.append(cat_data.std(axis=0).mean())
    ll[obsm] = np.mean(l)

In [16]:
ll

{'0.0': 0.083778314,
 '0.2': 0.08111936,
 '0.6': 0.058996618,
 '0.9': 0.03771937,
 '0.91': 0.03284199,
 '0.92': 0.039633993,
 '0.93': 0.031141141,
 '0.94': 0.02947674,
 '0.95': 0.03690366,
 '0.96': 0.024454094,
 '0.97': 0.025349513,
 '0.98': 0.01860594,
 '0.99': 0.02002851,
 '0.991': 0.020553796,
 '0.992': 0.020371025,
 '0.993': 0.018559048,
 '0.994': 0.018481618,
 '0.995': 0.017757984,
 '0.996': 0.022136897,
 '0.997': 0.01848099,
 '0.998': 0.020481892,
 '0.999': 0.023206994}

# Plot leidens

In [17]:
from tardis._disentanglementmanager import DisentanglementManager as DM

In [18]:
latent_dict.keys()

dict_keys(['scvi', 0.0, 0.2, 0.6, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 0.991, 0.992, 0.993, 0.994, 0.995, 0.996, 0.997, 0.998, 0.999])

In [19]:
adata = ad.read_h5ad(adata_file_path)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
sc.pp.pca(adata)

In [20]:
for k, v in latent_dict.items():
    adata.obsm[str(k)] = v
adata

AnnData object with n_obs × n_vars = 19053 × 5000
    obs: 'mouse', 'experiment', 'time_int', 'time_cat', 'zone', 'status_control'
    uns: 'log1p', 'pca'
    obsm: 'X_pca', 'scvi', '0.0', '0.2', '0.6', '0.9', '0.91', '0.92', '0.93', '0.94', '0.95', '0.96', '0.97', '0.98', '0.99', '0.991', '0.992', '0.993', '0.994', '0.995', '0.996', '0.997', '0.998', '0.999'
    varm: 'PCs'
    layers: 'counts'

In [ ]:
reses = list(np.round(np.linspace(2, 0, 81), 3)[:-1])
random_states = [10, 20, 30]
obs_keys = ["zone", "status_control", "time_cat"]

n_leidens = []
for obs_key in tqdm.notebook.tqdm(obs_keys, desc='target', position=0, leave=False):
    ind = DM.configurations.get_ordered_obs_key().index(obs_key)
    for obsm in tqdm.notebook.tqdm(calcs, desc=f'Calculations', position=1, leave=False):
        data = adata.obsm[str(obsm)][:, list(range(8 * ind, 8 * (1 + ind)))]
        __adata = ad.AnnData(X=data)
        for res in tqdm.notebook.tqdm(reses, desc='Resolutions', position=2, leave=False):
            for random_state in tqdm.notebook.tqdm(random_states, desc='Random States', position=3, leave=False):
                _adata = __adata.copy()
                sc.pp.neighbors(_adata, random_state=random_state)
                sc.tl.leiden(_adata, resolution=res, random_state=random_state+1)
                n_leiden = _adata.obs["leiden"].nunique()
                n_leidens.append([obs_key, obsm, res, n_leiden, random_state])
                del _adata
                gc.collect();
        del __adata
        gc.collect();

In [23]:
n_leiden_df = pd.DataFrame(n_leidens, columns=["target", "ratio_ablation", "resolution", "n_leiden", "random_state"])
n_leiden_df["ratio_ablation"] = n_leiden_df["ratio_ablation"].astype(str)
n_leiden_df.sort_values(by=["target", "ratio_ablation", "resolution"], ignore_index=True, inplace=True)
n_leiden_df.reset_index(drop=True, inplace=True)

In [24]:
adata.uns["ordered_targets"] = DM.configurations.get_ordered_obs_key()

In [25]:
ablation_adata_path = os.path.join(tardis.config.io_directories['figures'], "ablation_afriat.h5ad")
ablation_data_path = os.path.join(tardis.config.io_directories['figures'], "ablation_afriat.pickle")

In [26]:
# Bundle all objects into a single dictionary

data_to_save = {
    'n_leiden_df': n_leiden_df,
    'latent_dict': latent_dict,
    'reconst_r2_all': reconst_r2_all,
    'reconst_r2_valid': reconst_r2_valid,
    'mi_normalized': mi_normalized,
    'max_mig_all': max_mig_all,
    'max_mig_valid': max_mig_valid
}

# Save the dictionary into a pickle file
with open(ablation_data_path, 'wb') as file:
    pickle.dump(data_to_save, file)

adata.write_h5ad(ablation_adata_path)

In [27]:
# # Load the data from the pickle file
# with open(ablation_data_path, 'rb') as file:
#     data_loaded = pickle.load(file)
# 
# # Access your data
# n_leiden_df = data_loaded['n_leiden_df']
# latent_dict = data_loaded['latent_dict']
# reconst_r2_all = data_loaded['reconst_r2_all']
# reconst_r2_valid = data_loaded['reconst_r2_valid']
# mi_normalized = data_loaded['mi_normalized']
# max_mig_all = data_loaded['max_mig_all']
# max_mig_valid = data_loaded['max_mig_valid']
# 
# adata = ad.read_h5ad(ablation_adata_path)

In [28]:
# sns.lineplot(data=n_leiden_df, hue="ratio_ablation", y="n_leiden", x="resolution")
# plt.yscale("log")

In [29]:
# a metric to calculate the number of clusters e.g. it should be 3 in sample status etc.
# aws? knn etc?